# Problem Statement : From ImDb, extract user reviews of any 20 movies irrespective of the language and Perform sentiment analysis and LDA on the given data

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools
from imdb import IMDb


In [2]:
def getSoup(url):
    """
    Utility function which takes a url and returns a Soup object.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    return soup

In [3]:
#url = '''https://www.imdb.com/search/title/?genres=sci-fi&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=3396781f-d87f-4fac-8694-c56ce6f490fe&pf_rd_r=T3BPQAVH35K81EB1ABQB&pf_rd_s=center-1&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr1_i_2,desc&count=20'''

# get the soup object for main api url
#movies_soup = getSoup(url)


In [3]:
url = '''https://www.imdb.com/search/title/?title_type=feature&user_rating=4.0,10.0
&num_votes=50000,&genres=thriller&view=simple&sort=user_rating,desc&count=250'''

# get the soup object for main api url
movies_soup = getSoup(url)

In [4]:
# find all a-tags with class:None
movie_tags = movies_soup.find_all('a', attrs={'class': None})
#for tag in movie_tags:
        #print(tag.text)
# filter the a-tags to get just the titles
movie_tags = [tag.attrs['href'] for tag in movie_tags 
              if tag.attrs['href'].startswith('/title') & tag.attrs['href'].endswith('/')]

# remove duplicate links
movie_tags = list(dict.fromkeys(movie_tags))

#print("There are a total of " + str(len(movie_tags)) + " movie titles")
#print("Displaying 10 titles")
movie_tags = movie_tags[:20]

In [5]:
# movie links
base_url = "https://www.imdb.com"
movie_links = [base_url + tag + 'reviews' for tag in movie_tags]
movie_links[:10]
movie_links= movie_links[:20]

In [6]:
movie_links

['https://www.imdb.com/title/tt0468569/reviews',
 'https://www.imdb.com/title/tt1375666/reviews',
 'https://www.imdb.com/title/tt6751668/reviews',
 'https://www.imdb.com/title/tt0114369/reviews',
 'https://www.imdb.com/title/tt0102926/reviews',
 'https://www.imdb.com/title/tt0482571/reviews',
 'https://www.imdb.com/title/tt0407887/reviews',
 'https://www.imdb.com/title/tt0114814/reviews',
 'https://www.imdb.com/title/tt0110413/reviews',
 'https://www.imdb.com/title/tt0054215/reviews',
 'https://www.imdb.com/title/tt7286456/reviews',
 'https://www.imdb.com/title/tt0405094/reviews',
 'https://www.imdb.com/title/tt0364569/reviews',
 'https://www.imdb.com/title/tt0209144/reviews',
 'https://www.imdb.com/title/tt0051201/reviews',
 'https://www.imdb.com/title/tt0047396/reviews',
 'https://www.imdb.com/title/tt8579674/reviews',
 'https://www.imdb.com/title/tt8108198/reviews',
 'https://www.imdb.com/title/tt0105236/reviews',
 'https://www.imdb.com/title/tt0091251/reviews']

In [7]:
title=[]
movie_reviews=[]
for link in movie_links:
    user_reviews=[]
    soup = getSoup(link)
    title.append(soup.find('div',{'class' :'parent'}).find('a').text)
    for review in soup.find_all('div',{'class' :'text show-more__control'}):
        user_reviews.append(review.text)
    movie_reviews.append(user_reviews)
    
    

In [8]:
df = pd.DataFrame({'movie_names': title, 'movie_reviews': movie_reviews})
df

,movie_names,movie_reviews
0,The Dark Knight,"[Confidently directed, dark, brooding, and pac..."
1,Inception,"[My 3rd time watching this movie! Yet, it stil..."
2,Gisaengchung,[It's not really a review but my attempt to ex...
3,Se7en,"[The movie, ""Se7en"", starring Brad Pitt, Morga..."
4,The Silence of the Lambs,[The Silence of the Lambs runs two hours.Antho...
5,The Prestige,[Great movie with Great actors writers and twi...
6,The Departed,"[One of the best films of the 21st century, Th..."
7,The Usual Suspects,"[Ah, the Usual Suspects. My personal favorite ..."
8,Léon,"[You have to watch this film. It is beautiful,..."
9,Psycho,[No matter how many times one sits through thi...


In [9]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
print(nlp.pipe_names)
nlp.pipeline

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']


[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1a2d0fa40>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1a2d0fca8>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1a2a458d0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1a2a45a08>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1a2d0da88>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1a2d0d288>)]

In [10]:
def getKeywords(user_review):
    
    # create the doc object
    doc = nlp(user_review)
    print(doc.noun_chunks)
    # get a list of noun_chunks
    noun_chunks = [i for i in doc.noun_chunks]
    
    
    #convert noun_chunks from span objects to strings, otherwise it won't pickle
    noun_chunks_strlist = [chunk.text for chunk in noun_chunks]
    
    return noun_chunks_strlist

In [11]:
df['reviews']=df['movie_reviews'].astype(str)
df['keywords'] = df['reviews'].apply(getKeywords)


In [12]:
df

,movie_names,movie_reviews,reviews,keywords
0,The Dark Knight,"[Confidently directed, dark, brooding, and pac...","['Confidently directed, dark, brooding, and pa...","[impressive action sequences, a complex story,..."
1,Inception,"[My 3rd time watching this movie! Yet, it stil...","['My 3rd time watching this movie! Yet, it sti...","[this movie, it, my mind, me, its every moment..."
2,Gisaengchung,[It's not really a review but my attempt to ex...,['It\'s not really a review but my attempt to ...,"[It\, a review, my attempt, I, the movie, who,..."
3,Se7en,"[The movie, ""Se7en"", starring Brad Pitt, Morga...","['The movie, ""Se7en"", starring Brad Pitt, Morg...","[The movie, Se7en, Brad Pitt, Morgan Freeman, ..."
4,The Silence of the Lambs,[The Silence of the Lambs runs two hours.Antho...,['The Silence of the Lambs runs two hours.Anth...,"[The Silence, the Lambs, Anthony Hopkins, litt..."
5,The Prestige,[Great movie with Great actors writers and twi...,"[""Great movie with Great actors writers and tw...","[[""Great movie, Great actors writers, twists, ..."
6,The Departed,"[One of the best films of the 21st century, Th...","[""One of the best films of the 21st century, T...","[the best films, the 21st century, The Departe..."
7,The Usual Suspects,"[Ah, the Usual Suspects. My personal favorite ...","['Ah, the Usual Suspects. My personal favorite...","[['Ah, the Usual Suspects, My personal favorit..."
8,Léon,"[You have to watch this film. It is beautiful,...","[""You have to watch this film. It is beautiful...","[You, this film, It, no one, Mto, her man, She..."
9,Psycho,[No matter how many times one sits through thi...,['No matter how many times one sits through th...,"[this Hitchcock classic, Anthony Perkins, you,..."


In [14]:
import numpy as np
import itertools

def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 1)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([df['reviews'][0]])
candidates = count.get_feature_names()

In [16]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

HDS_keywords=[]
for index in df.index:
    doc_embedding = model.encode([df['reviews'][index]])
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([df['reviews'][index]])
    candidates = count.get_feature_names()
    #candidate_embeddings = model.encode(df['keywords'][index])
    candidate_embeddings = model.encode(candidates)
    HDS_keywords.append(max_sum_sim(doc_embedding, candidate_embeddings, df['keywords'][index], top_n=10, nr_candidates=20))
    


In [17]:
df['HDS_keywords']=HDS_keywords

In [18]:
df

,movie_names,movie_reviews,reviews,keywords,HDS_keywords
0,The Dark Knight,"[Confidently directed, dark, brooding, and pac...","['Confidently directed, dark, brooding, and pa...","[impressive action sequences, a complex story,...","[screen, the feel, His wide shots, the Joker, ..."
1,Inception,"[My 3rd time watching this movie! Yet, it stil...","['My 3rd time watching this movie! Yet, it sti...","[this movie, it, my mind, me, its every moment...","[The CGI, Joseph Gordon-Levitt, movies, it, my..."
2,Gisaengchung,[It's not really a review but my attempt to ex...,['It\'s not really a review but my attempt to ...,"[It\, a review, my attempt, I, the movie, who,...","[Mr. Park\'s house, the better endings, Ki-Tae..."
3,Se7en,"[The movie, ""Se7en"", starring Brad Pitt, Morga...","['The movie, ""Se7en"", starring Brad Pitt, Morg...","[The movie, Se7en, Brad Pitt, Morgan Freeman, ...","[He, the images, who, the fight, one point, pa..."
4,The Silence of the Lambs,[The Silence of the Lambs runs two hours.Antho...,['The Silence of the Lambs runs two hours.Anth...,"[The Silence, the Lambs, Anthony Hopkins, litt...","[the house chase, Roger Corman, I, brilliant e..."
5,The Prestige,[Great movie with Great actors writers and twi...,"[""Great movie with Great actors writers and tw...","[[""Great movie, Great actors writers, twists, ...","[they, it, grandma, the showy, Nolan's strengt..."
6,The Departed,"[One of the best films of the 21st century, Th...","[""One of the best films of the 21st century, T...","[the best films, the 21st century, The Departe...","[fantastic help, Boston, he, the movie, he, hi..."
7,The Usual Suspects,"[Ah, the Usual Suspects. My personal favorite ...","['Ah, the Usual Suspects. My personal favorite...","[['Ah, the Usual Suspects, My personal favorit...","[one plot, himself, the story, a strong clue, ..."
8,Léon,"[You have to watch this film. It is beautiful,...","[""You have to watch this film. It is beautiful...","[You, this film, It, no one, Mto, her man, She...","[a hitman, half-brother, (Natalie Portman, any..."
9,Psycho,[No matter how many times one sits through thi...,['No matter how many times one sits through th...,"[this Hitchcock classic, Anthony Perkins, you,...","[Greenlawn Cemetary"".f, you, Dennis Hopper, so..."


In [19]:
for key in df['HDS_keywords']:
    print (key)

['screen', 'the feel', 'His wide shots', 'the Joker', 'him', 'past roles', 'me', 'a smile', 'Gotham', 'every aspect']
['The CGI', 'Joseph Gordon-Levitt', 'movies', 'it', 'my review', 'our hero', 'it', 'the audience', 'a rather poor film', 'their trust']
["Mr. Park\\'s house", 'the better endings', 'Ki-Taek', '3 accounts', 'mystery', 'the answer', 'even film', 'best movies', 'I', 'a decent plot']
['He', 'the images', 'who', 'the fight', 'one point', 'part', 'who', 'the dark recess', 'his major break', 'who']
['the house chase', 'Roger Corman', 'I', 'brilliant eloquence', 'all the right elements', 'Hopkins', 'the greatest suspense films', ', "Great job', 'the horror genre', 'every scene']
['they', 'it', 'grandma', 'the showy', "Nolan's strengths", 'a convenient plot device', 'what', 'en masse', 'how two magicians', 'such an inspiration']
['fantastic help', 'Boston', 'he', 'the movie', 'he', 'his job', 'the 2000s', '(Wahlberg', 'Who', 'his films']
['one plot', 'himself', 'the story', 'a s

In [25]:
df.to_csv('keywordextracted.csv')

## SENTIMENT ANALYSIS

In [20]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(sentence):
  
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
  
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return ("Positive")
  
    elif sentiment_dict['compound'] <= - 0.05 :
        return("Negative")
  
    else :
        return("Neutral")

In [21]:
sentiments = []
movie_name=[]
reviewtext=[]

for index in df.index:
    emotion = []
    for reviews in df['movie_reviews'][index]:
        movie_name.append(df['movie_names'][index])
        reviewtext.append(reviews)
        sentiment = sentiment_scores(reviews)
        sentiments.append(sentiment)
        
  

In [22]:
df_final = pd.DataFrame({'movie_name': movie_name, 'reviewtext': reviewtext,'sentiments':sentiments})
df_final

,movie_name,reviewtext,sentiments
0,The Dark Knight,"Confidently directed, dark, brooding, and pack...",Positive
1,The Dark Knight,I got to see The Dark Knight on Wednesday nigh...,Positive
2,The Dark Knight,It is just what you want for the best movie. G...,Positive
3,The Dark Knight,We've been subjected to enormous amounts of hy...,Positive
4,The Dark Knight,This is the best superhero movie ever made!!!!...,Positive
...,...,...,...
494,Idi i smotri,This is just about as powerful as a war film c...,Positive
495,Idi i smotri,"Elem Klimov directs ""Come and See"". While the ...",Negative
496,Idi i smotri,I saw this movie many years ago and scenes sti...,Negative
497,Idi i smotri,Elem Klimov's COME AND SEE is a film best reme...,Negative


In [23]:
df_final.groupby('movie_name')['sentiments'].value_counts()

movie_name                   sentiments
1917                         Positive      22
                             Negative       3
Andhadhun                    Positive      21
                             Negative       2
                             Neutral        2
Gisaengchung                 Positive      20
                             Negative       5
Idi i smotri                 Negative      17
                             Positive       8
Inception                    Positive      23
                             Negative       2
Joker                        Positive      18
                             Negative       7
Léon                         Positive      23
                             Negative       2
Memento                      Positive      14
                             Negative      11
Oldeuboi                     Positive      17
                             Negative       8
Psycho                       Positive      14
                             Negative   

## LDA

In [24]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

/Users/poojaholkar/.pyenv/versions/3.7.3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [25]:
# Load the regular expression library
import re
# Remove punctuation
df_final['review_text_processed'] = \
df_final['reviewtext'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_final['review_text_processed'] = \
df_final['review_text_processed'].map(lambda x: x.lower())
df_final['review_text_processed'].head()

0    confidently directed dark brooding and packed ...
1    i got to see the dark knight on wednesday nigh...
2    it is just what you want for the best movie gr...
3    we've been subjected to enormous amounts of hy...
4    this is the best superhero movie ever made if ...
Name: review_text_processed, dtype: object

In [26]:
df_final

,movie_name,reviewtext,sentiments,review_text_processed
0,The Dark Knight,"Confidently directed, dark, brooding, and pack...",Positive,confidently directed dark brooding and packed ...
1,The Dark Knight,I got to see The Dark Knight on Wednesday nigh...,Positive,i got to see the dark knight on wednesday nigh...
2,The Dark Knight,It is just what you want for the best movie. G...,Positive,it is just what you want for the best movie gr...
3,The Dark Knight,We've been subjected to enormous amounts of hy...,Positive,we've been subjected to enormous amounts of hy...
4,The Dark Knight,This is the best superhero movie ever made!!!!...,Positive,this is the best superhero movie ever made if ...
...,...,...,...,...
494,Idi i smotri,This is just about as powerful as a war film c...,Positive,this is just about as powerful as a war film c...
495,Idi i smotri,"Elem Klimov directs ""Come and See"". While the ...",Negative,"elem klimov directs ""come and see"" while the f..."
496,Idi i smotri,I saw this movie many years ago and scenes sti...,Negative,i saw this movie many years ago and scenes sti...
497,Idi i smotri,Elem Klimov's COME AND SEE is a film best reme...,Negative,elem klimov's come and see is a film best reme...


In [39]:
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import nltk
from pprint import pprint
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
for movie in df_final['movie_name']:
    data = df_final[df_final['movie_name']== movie].review_text_processed.values.tolist()
    data_words = list(sent_to_words(data))
    # remove stop words
    data_words = remove_stopwords(data_words)
    # Create Dictionary
    id2word = corpora.Dictionary(data_words)
    # Create Corpus
    texts = data_words
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    # number of topics
    num_topics = 10
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
    # Print the Keyword in the 10 topics
    pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaholkar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[(0,
  '0.018*"best" + 0.018*"movie" + 0.015*"film" + 0.015*"batman" + '
  '0.014*"joker" + 0.014*"dark" + 0.012*"ledger" + 0.011*"heath" + '
  '0.010*"knight" + 0.008*"one"'),
 (1,
  '0.019*"movie" + 0.012*"batman" + 0.010*"superhero" + 0.010*"dark" + '
  '0.009*"watch" + 0.009*"one" + 0.008*"good" + 0.008*"best" + 0.008*"like" + '
  '0.007*"knight"'),
 (2,
  '0.011*"batman" + 0.009*"ledger" + 0.006*"heath" + 0.006*"bale" + '
  '0.006*"film" + 0.005*"joker" + 0.005*"though" + 0.005*"nolan" + '
  '0.005*"dark" + 0.005*"begins"'),
 (3,
  '0.011*"batman" + 0.010*"movie" + 0.008*"one" + 0.007*"dark" + 0.006*"joker" '
  '+ 0.006*"knight" + 0.006*"film" + 0.005*"go" + 0.005*"gotham" + '
  '0.005*"performance"'),
 (4,
  '0.017*"batman" + 0.010*"dark" + 0.009*"knight" + 0.008*"bale" + '
  '0.008*"ledger" + 0.007*"character" + 0.006*"movie" + 0.006*"film" + '
  '0.006*"could" + 0.006*"heath"'),
 (5,
  '0.014*"movie" + 0.012*"batman" + 0.009*"film" + 0.009*"one" + 0.009*"dark" '
  '+ 0.008*"kni

[(0,
  '0.015*"one" + 0.012*"film" + 0.012*"best" + 0.011*"knight" + 0.010*"movie" '
  '+ 0.010*"movies" + 0.010*"dark" + 0.009*"batman" + 0.008*"time" + '
  '0.006*"joker"'),
 (1,
  '0.025*"movie" + 0.013*"batman" + 0.011*"dark" + 0.010*"knight" + '
  '0.009*"ever" + 0.009*"best" + 0.008*"heath" + 0.008*"one" + 0.008*"much" + '
  '0.008*"ledger"'),
 (2,
  '0.024*"movie" + 0.012*"dark" + 0.011*"batman" + 0.011*"superhero" + '
  '0.010*"best" + 0.010*"watch" + 0.008*"one" + 0.008*"knight" + 0.007*"good" '
  '+ 0.007*"like"'),
 (3,
  '0.011*"ledger" + 0.010*"movie" + 0.010*"batman" + 0.009*"joker" + '
  '0.008*"heath" + 0.006*"villain" + 0.005*"knight" + 0.005*"dark" + '
  '0.005*"would" + 0.005*"hero"'),
 (4,
  '0.013*"film" + 0.013*"joker" + 0.010*"batman" + 0.009*"ledger" + '
  '0.008*"movie" + 0.008*"dark" + 0.007*"nolan" + 0.007*"character" + '
  '0.006*"heath" + 0.006*"knight"'),
 (5,
  '0.018*"batman" + 0.012*"knight" + 0.011*"dark" + 0.010*"movie" + '
  '0.008*"film" + 0.008*"led

[(0,
  '0.015*"movie" + 0.014*"batman" + 0.013*"joker" + 0.011*"heath" + '
  '0.010*"dark" + 0.009*"ledger" + 0.008*"knight" + 0.008*"best" + 0.007*"one" '
  '+ 0.006*"gotham"'),
 (1,
  '0.023*"movie" + 0.019*"batman" + 0.011*"dark" + 0.011*"film" + '
  '0.010*"knight" + 0.010*"best" + 0.010*"heath" + 0.009*"ledger" + '
  '0.008*"joker" + 0.008*"superhero"'),
 (2,
  '0.013*"movie" + 0.012*"batman" + 0.007*"joker" + 0.007*"ledger" + '
  '0.006*"go" + 0.006*"dark" + 0.005*"heath" + 0.005*"night" + 0.005*"best" + '
  '0.005*"would"'),
 (3,
  '0.013*"dark" + 0.013*"batman" + 0.011*"movie" + 0.009*"knight" + '
  '0.008*"one" + 0.007*"well" + 0.007*"best" + 0.006*"gotham" + 0.006*"good" + '
  '0.006*"heath"'),
 (4,
  '0.017*"movie" + 0.012*"dark" + 0.011*"much" + 0.011*"batman" + '
  '0.009*"knight" + 0.008*"see" + 0.008*"seen" + 0.007*"one" + 0.007*"joker" + '
  '0.006*"heath"'),
 (5,
  '0.010*"dark" + 0.010*"batman" + 0.010*"ledger" + 0.007*"film" + '
  '0.006*"good" + 0.006*"bale" + 0.006

[(0,
  '0.025*"movie" + 0.015*"batman" + 0.014*"best" + 0.011*"superhero" + '
  '0.011*"knight" + 0.011*"dark" + 0.010*"one" + 0.009*"joker" + 0.008*"like" '
  '+ 0.007*"heath"'),
 (1,
  '0.023*"batman" + 0.012*"movie" + 0.012*"dark" + 0.011*"best" + '
  '0.010*"ledger" + 0.010*"joker" + 0.009*"heath" + 0.009*"knight" + '
  '0.008*"bale" + 0.006*"performance"'),
 (2,
  '0.017*"movie" + 0.014*"film" + 0.011*"one" + 0.010*"dark" + 0.010*"much" + '
  '0.009*"knight" + 0.009*"batman" + 0.009*"time" + 0.008*"see" + '
  '0.007*"joker"'),
 (3,
  '0.010*"batman" + 0.007*"heath" + 0.007*"dark" + 0.006*"film" + '
  '0.005*"ledger" + 0.004*"nolan" + 0.004*"knight" + 0.004*"one" + '
  '0.003*"movie" + 0.003*"joker"'),
 (4,
  '0.013*"movie" + 0.013*"batman" + 0.012*"ledger" + 0.012*"dark" + '
  '0.011*"joker" + 0.009*"heath" + 0.009*"knight" + 0.008*"best" + '
  '0.006*"film" + 0.006*"character"'),
 (5,
  '0.010*"dark" + 0.008*"batman" + 0.007*"knight" + 0.006*"movie" + '
  '0.005*"one" + 0.005*"ba

...

[(0,
  '0.015*"war" + 0.012*"film" + 0.012*"see" + 0.011*"one" + 0.008*"come" + '
  '0.008*"movie" + 0.006*"like" + 0.006*"florya" + 0.005*"ever" + '
  '0.005*"time"'),
 (1,
  '0.012*"war" + 0.011*"film" + 0.011*"see" + 0.009*"come" + 0.006*"old" + '
  '0.006*"years" + 0.006*"one" + 0.005*"movie" + 0.004*"many" + 0.004*"us"'),
 (2,
  '0.011*"war" + 0.010*"see" + 0.010*"film" + 0.009*"one" + 0.008*"come" + '
  '0.007*"movie" + 0.005*"like" + 0.005*"klimov" + 0.005*"much" + 0.004*"old"'),
 (3,
  '0.011*"film" + 0.010*"see" + 0.008*"movie" + 0.007*"war" + 0.007*"come" + '
  '0.007*"one" + 0.006*"like" + 0.005*"even" + 0.004*"klimov" + 0.003*"films"'),
 (4,
  '0.013*"film" + 0.011*"come" + 0.010*"see" + 0.010*"war" + 0.007*"movie" + '
  '0.005*"one" + 0.004*"even" + 0.004*"like" + 0.003*"florya" + 0.003*"films"'),
 (5,
  '0.014*"film" + 0.012*"see" + 0.011*"war" + 0.008*"come" + 0.006*"one" + '
  '0.004*"florya" + 0.004*"even" + 0.004*"movie" + 0.004*"many" + '
  '0.004*"klimov"'),
 (6,
  

[(0,
  '0.016*"war" + 0.013*"film" + 0.012*"see" + 0.010*"movie" + 0.008*"come" + '
  '0.007*"one" + 0.007*"village" + 0.006*"florya" + 0.005*"klimov" + '
  '0.004*"german"'),
 (1,
  '0.010*"war" + 0.009*"film" + 0.008*"see" + 0.006*"come" + 0.005*"florya" + '
  '0.005*"one" + 0.004*"movie" + 0.003*"many" + 0.003*"like" + 0.003*"films"'),
 (2,
  '0.020*"film" + 0.016*"see" + 0.013*"war" + 0.012*"come" + 0.009*"one" + '
  '0.005*"films" + 0.005*"us" + 0.004*"klimov" + 0.004*"seen" + 0.004*"movie"'),
 (3,
  '0.015*"film" + 0.009*"see" + 0.007*"come" + 0.006*"one" + 0.005*"war" + '
  '0.004*"florya" + 0.004*"klimov" + 0.004*"years" + 0.003*"old" + '
  '0.003*"much"'),
 (4,
  '0.013*"movie" + 0.009*"war" + 0.009*"film" + 0.008*"like" + 0.007*"one" + '
  '0.007*"see" + 0.006*"come" + 0.005*"florya" + 0.004*"time" + '
  '0.004*"director"'),
 (5,
  '0.014*"war" + 0.013*"see" + 0.012*"movie" + 0.012*"come" + 0.011*"film" + '
  '0.007*"one" + 0.005*"like" + 0.005*"even" + 0.004*"old" + 0.004*"y

[(0,
  '0.017*"war" + 0.015*"movie" + 0.009*"one" + 0.009*"film" + 0.007*"see" + '
  '0.006*"like" + 0.006*"come" + 0.005*"village" + 0.005*"years" + '
  '0.005*"soviet"'),
 (1,
  '0.015*"film" + 0.010*"come" + 0.010*"see" + 0.008*"one" + 0.007*"war" + '
  '0.006*"movie" + 0.004*"klimov" + 0.004*"seen" + 0.004*"florya" + '
  '0.004*"films"'),
 (2,
  '0.020*"film" + 0.018*"see" + 0.014*"come" + 0.011*"war" + 0.006*"one" + '
  '0.005*"klimov" + 0.005*"movie" + 0.004*"many" + 0.004*"florya" + '
  '0.004*"films"'),
 (3,
  '0.009*"film" + 0.008*"war" + 0.008*"see" + 0.006*"come" + 0.005*"like" + '
  '0.005*"movie" + 0.004*"one" + 0.003*"events" + 0.003*"village" + '
  '0.003*"years"'),
 (4,
  '0.011*"war" + 0.009*"film" + 0.007*"see" + 0.005*"come" + 0.005*"one" + '
  '0.004*"movie" + 0.004*"like" + 0.004*"even" + 0.003*"florya" + '
  '0.003*"made"'),
 (5,
  '0.020*"film" + 0.016*"see" + 0.015*"war" + 0.011*"one" + 0.010*"come" + '
  '0.006*"movie" + 0.006*"florya" + 0.006*"ever" + 0.005*"f

[(0,
  '0.008*"film" + 0.007*"war" + 0.006*"come" + 0.006*"see" + 0.006*"one" + '
  '0.005*"movie" + 0.004*"old" + 0.004*"like" + 0.003*"years" + 0.003*"even"'),
 (1,
  '0.020*"film" + 0.015*"see" + 0.012*"war" + 0.011*"come" + 0.011*"one" + '
  '0.009*"movie" + 0.008*"florya" + 0.006*"like" + 0.005*"films" + '
  '0.005*"klimov"'),
 (2,
  '0.008*"war" + 0.008*"film" + 0.007*"come" + 0.007*"one" + 0.006*"see" + '
  '0.004*"old" + 0.004*"films" + 0.003*"klimov" + 0.003*"years" + '
  '0.003*"soviet"'),
 (3,
  '0.018*"see" + 0.017*"film" + 0.014*"war" + 0.012*"come" + 0.008*"one" + '
  '0.005*"klimov" + 0.005*"movie" + 0.005*"like" + 0.005*"even" + '
  '0.004*"florya"'),
 (4,
  '0.014*"film" + 0.010*"war" + 0.009*"see" + 0.007*"come" + 0.006*"one" + '
  '0.005*"florya" + 0.004*"soldiers" + 0.004*"klimov" + 0.004*"really" + '
  '0.004*"movie"'),
 (5,
  '0.013*"war" + 0.011*"film" + 0.008*"come" + 0.008*"see" + 0.007*"one" + '
  '0.005*"movie" + 0.005*"years" + 0.004*"village" + 0.004*"old" 

[(0,
  '0.022*"film" + 0.013*"see" + 0.010*"come" + 0.008*"war" + 0.007*"one" + '
  '0.005*"movie" + 0.005*"ever" + 0.004*"though" + 0.004*"even" + '
  '0.004*"seen"'),
 (1,
  '0.015*"war" + 0.013*"film" + 0.012*"see" + 0.011*"come" + 0.009*"one" + '
  '0.009*"florya" + 0.006*"movie" + 0.005*"village" + 0.004*"like" + '
  '0.004*"old"'),
 (2,
  '0.014*"see" + 0.014*"war" + 0.012*"film" + 0.010*"come" + 0.010*"one" + '
  '0.006*"movie" + 0.005*"films" + 0.004*"like" + 0.004*"klimov" + '
  '0.004*"made"'),
 (3,
  '0.017*"film" + 0.013*"see" + 0.010*"war" + 0.008*"come" + 0.008*"movie" + '
  '0.007*"like" + 0.006*"one" + 0.005*"klimov" + 0.004*"florya" + '
  '0.004*"films"'),
 (4,
  '0.015*"war" + 0.012*"film" + 0.009*"one" + 0.008*"see" + 0.007*"come" + '
  '0.005*"movie" + 0.005*"florya" + 0.004*"ever" + 0.004*"time" + '
  '0.003*"klimov"'),
 (5,
  '0.011*"see" + 0.010*"war" + 0.009*"film" + 0.009*"come" + 0.006*"movie" + '
  '0.005*"florya" + 0.005*"one" + 0.005*"like" + 0.004*"klimov"